![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

In [19]:
# Load dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None)
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


In [21]:
cc_apps[cc_apps[0] == '?']

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
248,?,24.50,12.750,u,g,c,bb,4.750,t,t,2,g,444,+
327,?,40.83,3.500,u,g,i,bb,0.500,f,f,0,s,0,-
346,?,32.25,1.500,u,g,c,v,0.250,f,f,0,g,122,-
374,?,28.17,0.585,u,g,aa,v,0.040,f,f,0,g,1004,-
453,?,29.75,0.665,u,g,w,v,0.250,f,f,0,g,0,-
479,?,26.50,2.710,y,p,?,?,0.085,f,f,0,s,0,-
489,?,45.33,1.000,u,g,q,v,0.125,f,f,0,g,0,-
520,?,20.42,7.500,u,g,k,v,1.500,t,t,1,g,234,+
598,?,20.08,0.125,u,g,q,v,1.000,f,t,1,g,768,+
601,?,42.25,1.750,y,p,?,?,0.000,f,f,0,g,1,-


In [22]:
cc_apps[0].value_counts()

0
b    468
a    210
?     12
Name: count, dtype: int64

In [23]:
# Replace the '?'s with NaN in dataset
cc_apps_nans_replaced = cc_apps.replace("?", np.nan)

In [24]:
cc_apps_nans_replaced.isna().sum()

0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13     0
dtype: int64

In [30]:
cc_apps_nans_replaced[0].value_counts()

0
b    468
a    210
Name: count, dtype: int64

In [26]:
# Create a copy of the NaN replacement DataFrame
cc_apps_imputed = cc_apps_nans_replaced.copy()

In [27]:
# Iterate over each column of cc_apps_nans_replaced and impute the most frequent value for object data types and the mean for numeric data types
for col in cc_apps_imputed.columns:
    # Check if the column is of object type
    if cc_apps_imputed[col].dtypes == "object":
        # Impute with the most frequent value
        cc_apps_imputed[col] = cc_apps_imputed[col].fillna(
            cc_apps_imputed[col].value_counts().index[0]
        )
    else:
        cc_apps_imputed[col] = cc_apps_imputed[col].fillna(cc_apps_imputed[col].mean())

In [29]:
cc_apps_imputed[0].value_counts()

0
b    480
a    210
Name: count, dtype: int64

In [32]:
# Dummify the categorical features
cc_apps_encoded = pd.get_dummies(cc_apps_imputed, dtype='int', drop_first=True)

In [33]:
cc_apps_encoded

,2,7,10,12,0_b,1_15.17,1_15.75,1_15.83,1_15.92,1_16.00,...,6_j,6_n,6_o,6_v,6_z,8_t,9_t,11_p,11_s,13_-
0,0.000,1.25,1,0,1,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0
1,4.460,3.04,6,560,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
2,0.500,1.50,0,824,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,1.540,3.75,5,3,1,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0
4,5.625,1.71,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,10.085,1.25,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
686,0.750,2.00,2,394,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1
687,13.500,2.00,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
688,0.205,0.04,0,750,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [34]:
# Extract the last column as your target variable
X = cc_apps_encoded.iloc[:, :-1].values
y = cc_apps_encoded.iloc[:, [-1]].values

In [35]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [38]:
# Instantiate StandardScaler and use it to rescale X_train and X_test
scaler = StandardScaler()
X_trainrescaled = scaler.fit_transform(X_train)
X_testrescaled = scaler.transform(X_test)

In [39]:
# Instantiate a LogisticRegression classifier with default parameter values
logreg = LogisticRegression()

# Fit logreg to the train set
logreg.fit(X_trainrescaled, y_train)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [40]:
# Use logreg to predict instances from the training set
y_train_pred = logreg.predict(X_trainrescaled)

In [41]:
# Print the confusion matrix of the logreg model
print(confusion_matrix(y_train, y_train_pred))

[[203   1]
 [  1 257]]


In [42]:
# Define the grid of values for tol and max_iter
tol = [0.01, 0.001, 0.0001]
max_iter = [100, 150, 200]

In [43]:
# Create a dictionary where tol and max_iter are keys and the lists of their values are the corresponding values
param_grid = dict(tol=tol, max_iter=max_iter)

In [44]:
# Instantiate GridSearchCV with the required parameters
grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)

In [46]:
# Fit grid_model to the data
grid_model_result = grid_model.fit(X_trainrescaled, y_train.ravel())

In [47]:
# Summarize results
best_train_score, best_train_params = grid_model_result.best_score_, grid_model_result.best_params_
print("Best: %f using %s" % (best_train_score, best_train_params))

Best: 0.818256 using {'max_iter': 100, 'tol': 0.01}


In [48]:
# Extract the best model and evaluate it on the test set
best_model = grid_model_result.best_estimator_
best_score =  best_model.score(X_testrescaled, y_test)

In [49]:
print(best_model)
print(best_score)

LogisticRegression(tol=0.01)
0.7982456140350878


In [50]:
print("Accuracy of logistic regression classifier: ", best_score)

Accuracy of logistic regression classifier:  0.7982456140350878


In [51]:
best_score =  0.7982456140350878